# 毕业论文`GRAD`
## 建立多元线性回归模型，定量评估影响

*`Evan`*\
*`2023-11-16`*
---

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

import sys
sys.path.append('../../src/')
from namelist import *
from RandomForest import write_nc_to_df

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [2]:
lowyears  = [2014,2015,2016]
highyears = [2019,2021,2022]
month     = 'Sep'

# 设置从mcip和chem文件中读取的变量
mcip_variants = ['SFC_TMP', 'SOL_RAD', 'RH', 'PRES', 
                 'WSPD10', 'WDIR10','CloudFRAC','PBLH']
chem_variants = ['O3','NO2', 'VOC', 'PM25', 'ISOP']

# 设置输入rf模型的自变量列表和目标变量
variants = ['SFC_TMP', 'SOL_RAD', 'RH', 'PRES', 
            'WSPD10', 'WDIR10','CloudFRAC',]
target = 'O3'

In [3]:
dfs = write_nc_to_df(lowyears, month, processed_dir,
                     mcip_variants, chem_variants, variants, target,)

nx = 152, ny = 110
The process is expected to take up to 1839.20 min, or 30.65 h
(0,0) --> 0.01 %
(1,0) --> 0.01 %
(2,0) --> 0.02 %
(3,0) --> 0.02 %
(4,0) --> 0.03 %
(5,0) --> 0.04 %
(6,0) --> 0.04 %
(7,0) --> 0.05 %
(8,0) --> 0.05 %
(9,0) --> 0.06 %
(10,0) --> 0.07 %
(11,0) --> 0.07 %
(12,0) --> 0.08 %
(13,0) --> 0.08 %
(14,0) --> 0.09 %
(15,0) --> 0.10 %
(16,0) --> 0.10 %
(17,0) --> 0.11 %
(18,0) --> 0.11 %
(19,0) --> 0.12 %
(20,0) --> 0.13 %
(21,0) --> 0.13 %
(22,0) --> 0.14 %
(23,0) --> 0.14 %
(24,0) --> 0.15 %
(25,0) --> 0.16 %
(26,0) --> 0.16 %
(27,0) --> 0.17 %
(28,0) --> 0.17 %
(29,0) --> 0.18 %
(30,0) --> 0.19 %
(31,0) --> 0.19 %
(32,0) --> 0.20 %
(33,0) --> 0.20 %
(34,0) --> 0.21 %
(35,0) --> 0.22 %
(36,0) --> 0.22 %
(37,0) --> 0.23 %
(38,0) --> 0.23 %
(39,0) --> 0.24 %
(40,0) --> 0.25 %
(41,0) --> 0.25 %
(42,0) --> 0.26 %
(43,0) --> 0.26 %
(44,0) --> 0.27 %
(45,0) --> 0.28 %
(46,0) --> 0.28 %
(47,0) --> 0.29 %
(48,0) --> 0.29 %
(49,0) --> 0.30 %
(50,0) --> 0.31 %
(51,0) --> 

## 将`Dataframe`转为`Dataset`

In [6]:
dfs.keys()

dict_keys(['SFC_TMP', 'SOL_RAD', 'RH', 'PRES', 'WSPD10', 'WDIR10', 'CloudFRAC', 'mse', 'r2'])

In [10]:
dfs['SFC_TMP'].values.shape

(110, 152)

In [11]:
grid = xr.open_dataset(grid_d03)
lon = grid.LON.squeeze().values
lat = grid.LAT.squeeze().values

dataset = xr.Dataset(
    data_vars=dict(
        SFC_TMP   = (['y','x'],dfs['SFC_TMP'].values.astype(float)),
        SOL_RAD   = (['y','x'],dfs['SOL_RAD'].values.astype(float)),
        RH        = (['y','x'],dfs['RH'].values.astype(float)),
        PRES      = (['y','x'],dfs['PRES'].values.astype(float)),
        WSPD10    = (['y','x'],dfs['WSPD10'].values.astype(float)),
        WDIR10    = (['y','x'],dfs['WDIR10'].values.astype(float)),
        CloudFRAC = (['y','x'],dfs['CloudFRAC'].values.astype(float)),
        mse       = (['y','x'],dfs['mse'].values.astype(float)),
        r2        = (['y','x'],dfs['r2'].values.astype(float)),
    ),
    coords=dict(
        latitude = (['y','x'],lat),
        longitude = (['y','x'],lon),
    ),
)
dataset

<xarray.Dataset>
Dimensions:    (y: 110, x: 152)
Coordinates:
    latitude   (y, x) float32 21.54 21.54 21.54 21.55 ... 24.57 24.57 24.57
    longitude  (y, x) float32 111.1 111.2 111.2 111.2 ... 115.6 115.6 115.7
Dimensions without coordinates: y, x
Data variables:
    SFC_TMP    (y, x) float64 0.126 0.1261 0.1145 ... 0.1129 0.1126 0.1174
    SOL_RAD    (y, x) float64 0.07103 0.06964 0.0889 ... 0.2262 0.1937 0.07311
    RH         (y, x) float64 0.3448 0.3392 0.3278 ... 0.1124 0.1341 0.2153
    PRES       (y, x) float64 0.1179 0.1314 0.1353 ... 0.1269 0.129 0.1474
    WSPD10     (y, x) float64 0.08682 0.079 0.08793 ... 0.135 0.1173 0.1203
    WDIR10     (y, x) float64 0.1628 0.1633 0.1463 ... 0.1788 0.1974 0.1963
    CloudFRAC  (y, x) float64 0.09067 0.09146 0.09919 ... 0.1077 0.1159 0.1302
    mse        (y, x) float64 681.1 676.0 689.2 626.1 ... 376.8 358.9 343.0
    r2         (y, x) float64 0.5605 0.5714 0.5675 ... 0.4869 0.4796 0.4721

In [12]:
outpath = 'D:/data/Graduation/Contribution/RandomForest_output/'
dataset.to_netcdf(outpath + 'RF_grids_low.nc')